In [ ]:
!wget -nc -q https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat
!wget -nc -q https://raw.githubusercontent.com/jpatokal/openflights/master/data/routes.dat

In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import geopy.distance
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoDataFrame

## Reading and converting data format

We start by reading the airports file to a pandas Dataframe.

In [ ]:
airportsDF = pd.read_csv('airports.dat', header=None, names=['id', 'name', 'city', 'country', 'IATA', 'ICAO', 'lat', 'lon', 'alt', 'timezone', 'DST', 'Tz', 'type', 'source'])

From there, we create a directed graph and add all the airports as nodes to it (except airports with no IATA identifier, which are ignored).
This is because we will use this identifiers later to associate this data with the routes data.

In [ ]:
G = nx.DiGraph()

for index, row in airportsDF.iterrows():
    if row['IATA'] == '\\N':
        continue
    G.add_node(row['IATA'], pos=(row['lat'],row['lon']), country=row['country'])

Our nodes are identified by their IATA identifiers and have a corresponding position (corresponding to their real geographic location) 

In [ ]:
print(G.nodes['LIS'])

So far we have a graph with nodes but no edges

In [ ]:
print("Node count: " + str(len(G.nodes)))
print("Edge count: " + str(len(G.edges)))

Now we read the routes data and add each one as an edge of our graph previously created
This routes are directed, meaning that if an airline operates services from A to B and from B to A, both A-B and B-A are listed as diferent edges
We also add weight to this edges, the distance between airports in km.

In [ ]:
routesDF = pd.read_csv('routes.dat', header=None, names=['airline', 'airlineID', 'source', 'sourceID', 'dest', 'destID', 'codeshare', 'stops', 'equipment'])

for index, row in routesDF.iterrows():
    if(row['source'] not in G.nodes or row['dest'] not in G.nodes):
        continue
    dist = geopy.distance.distance((G.nodes[row['source']]['pos']), (G.nodes[row['dest']]['pos'])).km
    G.add_edge(row['source'], row['dest'], weight=dist)

As an example, we take a route from Lisbon to Cologne to show this representation.

In [ ]:
print(list(G.edges('LIS',data=True))[0])

We can now see that our graph has nodes and edges.

In [ ]:
print("Node count: " + str(len(G.nodes)))
print("Edge count: " + str(len(G.edges)))

## Cleaning data

We now "clean" our graph, removing the nodes which have either 0 incoming flights or 0 outgoing flights (or both).
We iterate the graph until no more changes are made. This is because after removing one node, another one might lose the only incoming or outgoing route it had previously.

In [ ]:
i=0
while i == 0:
  i = 1
  H = G.copy()
  for node in H.in_degree:
    if node[1] == 0:
      i = 0
      G.remove_node(node[0])
      
  H = G.copy()
  for node in H.out_degree:
    if node[1] == 0:
      i = 0
      G.remove_node(node[0])

We end up with a significant reduction in nodes and lose barely any edges, obtaining what we wanted and removing unutilized airports from the network.

In [ ]:
print("Node count: " + str(len(G.nodes)))
print("Edge count: " + str(len(G.edges)))

## Analysing network metrics

We can also see that the Frankfurt airport, Charles de Gaulle airport in Paris and Amesterdam are the nodes with the highest degree, with around 450 incoming or outgoing routes each.

In [ ]:
nodes = sorted(G.degree, key=lambda x: x[1], reverse=True)
print(nodes[0:3])

We also see that the majority of airports have a very low degree, resulting in a near zero density in our graph.

In [ ]:
# Graph density
nx.density(G)

With the degree distribution we verify that almost 800 airports (of the near 3000) have a degree of only 2, meaning only one incoming route and one outgoing one.

In [ ]:
# Degree Distribution
degrees = [G.degree(n) for n in G.nodes()]
plt.hist(degrees, bins=np.logspace(np.log10(2),np.log10(max(degrees)), 20))
plt.xscale("log")
plt.title("Histogram of Degree Distribution")
plt.ylabel("Node Count")
plt.xlabel("Degree Value")


The comulative distribution also proves this point.

In [ ]:
import collections

degrees = [G.degree(n) for n in G.nodes()]
degree_sequence = sorted(degrees, reverse=True) # degree sequence
degreeCount = collections.Counter(degree_sequence)
deg, cnt = zip(*degreeCount.items())
cs = np.cumsum(cnt)
plt.loglog(deg, cs, 'bo')
plt.title("Cumulative Distribution plot")
plt.ylabel("Sample with value > Degree")
plt.xlabel("Degree")
plt.show()

## Degree centrality

We now calculate some metrics of the network, starting with the degree centrality
As seen before, Frankfurt is the airport with the highest degree.
A military base in Greenland is the one of the many airports with only degree 2, resulting in a low degree centrality.

In [ ]:
# compute graph degree centrality
dc = nx.degree_centrality(G)

max_dc = max(dc, key=dc.get)
print("Airport with the max degree centrality: " + max_dc)
print("Value: " + str(dc[max_dc]))

min_dc = min(dc, key=dc.get)
print("Airport with the min degree centrality: " + min_dc)
print("Value: " + str(dc[min_dc]))

# get top 5
top5 = sorted(dc, key=dc.get, reverse=True)[:5]
for airport in top5:
    print(airport + ": " + str(dc[airport])+ " " + G.nodes[airport]['country'])



## Eigenvector centrality

For the eigenvector centrality we get that Amsterdam airport has the highest value.

In [ ]:
# compute eigen vector centrality
ec = nx.eigenvector_centrality(G)

max_ec = max(ec, key=ec.get)
print("Airport with the max eigenvector centrality: " + max_ec)
print("Value: " + str(ec[max_ec]))

min_ec = min(ec, key=ec.get)
print("Airport with the min eigenvector centrality: " + min_ec)
print("Value: " + str(ec[min_ec]))


## Closeness centrality

For the closeness centrality we get that Frankfurt airport has the highest value.

In [ ]:
# compute closeness centrality
cc = nx.closeness_centrality(G, distance='weight')

max_cc = max(cc, key=cc.get)
print("Airport with the max closeness centrality: " + max_cc)
print("Value: " + str(cc[max_cc]))

min_cc = min(cc, key=cc.get)
print("Airport with the min closeness centrality: " + min_cc)
print("Value: " + str(cc[min_cc]))

# compute closeness centrality mean
cc_mean = sum(cc.values())/len(cc)
print("Mean closeness centrality: " + str(cc_mean))

# compute mode of closeness centrality
cc_mode = max(set(cc.values()), key=list(cc.values()).count)
print("Mode of closeness centrality: " + str(cc_mode))

# compute standard deviation of closeness centrality
cc_std = np.std(list(cc.values()))
print("Standard deviation of closeness centrality: " + str(cc_std))



## Harmonic centrality

In [ ]:
# compute harmonic centrality
hc = nx.harmonic_centrality(G, distance='weight')

max_hc = max(hc, key=hc.get)
print("Airport with the max harmonic centrality: " + max_hc)
print("Value: " + str(hc[max_hc]))

min_hc = min(hc, key=hc.get)
print("Airport with the min harmonic centrality: " + min_hc)
print("Value: " + str(hc[min_hc]))


## Betweenness centrality

For the betweenness centrality we get that Charles de Gaulle airport has the highest value.

In [ ]:
# compute betweenness centrality
bc = nx.betweenness_centrality(G)

max_bc = max(bc, key=bc.get)
print("Airport with the max betweenness centrality: " + max_bc)
print("Value: " + str(bc[max_bc]))

min_bc = min(bc, key=bc.get)
print("Airport with the min betweenness centrality: " + min_bc)
print("Value: " + str(bc[min_bc]))

# sort the betweenness centrality
sorted_bc = sorted(bc.items(), key=lambda x: x[1], reverse=True)
print(sorted_bc[0:5])

## Page rank

For the page rank we get that Charles de Gaulle airport has the highest value.

In [ ]:
# compute pagerank
pr = nx.pagerank(G)

max_pr = max(pr, key=pr.get)
print("Airport with the max page rank: " + max_pr)
print("Value: " + str(pr[max_pr]))

min_pr = min(pr, key=pr.get)
print("Airport with the min page rank: " +min_pr)
print("Value: " + str(pr[min_pr]))

# sort page rank
sorted_pr = sorted(pr.items(), key=lambda x: x[1], reverse=True)
print(sorted_pr[0:5])


In [ ]:
# plot 4 histograms with the 4 centrality measures
plt.figure(figsize=(20,10))
plt.subplot(2,2,1)
plt.hist(list(dc.values()), bins=20)
plt.title("Degree Centrality")
plt.subplot(2,2,2)
plt.hist(list(ec.values()), bins=20)
plt.title("Eigenvector Centrality")
plt.subplot(2,2,3)
plt.hist(list(cc.values()), bins=20)
plt.title("Closeness Centrality")
plt.subplot(2,2,4)
plt.hist(list(bc.values()), bins=20)
plt.title("Betweenness Centrality")
plt.show()

## Clustering

For the clustring we get that Goroka airport has the highest value. 
### TODO: check why

In [ ]:
# compute clustering coefficient
cl = nx.clustering(G)

max_cl = max(cl, key=cl.get)
print("Airport with the max clustering: " + max_cl)
print("Value: " + str(cl[max_cl]))

min_cl = min(cl, key=cl.get)
print("Airport with the min clustering: " + min_cl)
print("Value: " + str(cl[min_cl]))

### Strongly connected components

Using the networkx function to get us the strongly connected components, we verify that out graph has 7.
One of them has almost every node and the rest are small, due to this we'll only consider the biggest one in some future steps.

In [ ]:
# generate strongly connected components
sccs = nx.strongly_connected_components(G)
print([len(Gc) for Gc in sccs])

sccs = nx.strongly_connected_components(G)
sccs_graphs = (G.subgraph(c) for c in sccs)

largest_scc = list(sccs_graphs)[0]
print(len(largest_scc))

Before having a strongly connected component, calculating the average shortest path and the diameter of the network was not possible since there were some infinite distances, now we can do it.
## TODO remove nodes with low degree and check this metrics

In [ ]:
# average shortest path
#TODO: add weights

#avg_path_length = nx.average_shortest_path_length(largest_scc, weight='weight')
#sprint(avg_path_length)

In [ ]:
#nx.diameter(largest_scc)

## Plotting the data on a map

We can now plot each airport in a map and give each dot some coloring and size according to its centrality.
Having created a function that receives which centrality to be used, the list of nodes to be plotted and a threshold (values lower that this threshold won't be displayed), we can plot the maps for each of the centralities calculated before.

In [ ]:
def plotMap(label, centrality_dict, list_nodes, threshold):
    newAirportsDF = pd.DataFrame(columns=['id', 'name', 'city', 'country', 'IATA', 'ICAO', 'lat', 'lon', 'alt', 'timezone', 'DST', 'Tz', 'type', 'source'])
    
    # get max and min to normalize
    max_value = max(centrality_dict.values())
    min_value = min(centrality_dict.values())

    # from the list given, add the nodes that have a higher centrality value than the threshold
    for node in list_nodes:
        if(centrality_dict[node] > threshold):
            x = airportsDF.loc[airportsDF['IATA'] == node]
            x_norm = (centrality_dict[node] - min_value) / (max_value - min_value)
            x['color'] = centrality_dict[node]
            x['markersize'] = (x_norm **4) * 50
            newAirportsDF = pd.concat([newAirportsDF, x])

    geometry = [Point(xy) for xy in zip(newAirportsDF['lon'], newAirportsDF['lat'])]
    gdf = GeoDataFrame(newAirportsDF, geometry=geometry)

    # #this is a simple map that goes with geopandas
    world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
    gdf.plot(column="color", ax=world.plot(figsize=(20, 12)), marker='o', markersize="markersize", cmap='rainbow') 
            #,legend=True, legend_kwds={"label": label, "orientation": "horizontal"})

In [ ]:
# Ignores a warning that is not worrying in this case and creates a lot of noisy output
import warnings
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [ ]:
D = G.copy()
print(len(D.nodes))
print(len(D.edges))
i=0
while i == 0:
  i = 1
  H = D.copy()
  for node in H.in_degree:
    if node[1] < 5:
      i = 0
      D.remove_node(node[0])
      
  H = D.copy()
  for node in H.out_degree:
    if node[1] < 5:
      i = 0
      D.remove_node(node[0])

print(len(D.nodes))
nodes = sorted(D.degree, key=lambda x: x[1])
print(nodes)
print(len(D.edges))

## Degree centrality map

In [ ]:
plotMap("Degree centrality", dc, G.nodes, 0)

## Eigenvector centrality map

In [ ]:
plotMap("Eigenvector centrality", ec, D.nodes, 0)

## Closeness centrality map

In [ ]:
plotMap("Closeness centrality", cc, D.nodes, 0)

## Harmonic Centrality Map

In [ ]:
plotMap("Harmonic Ceentrality", hc, D.nodes, 0)

## Betweenness centrality map

In [ ]:
plotMap("Betweenness centrality", bc, D.nodes, 0)

## Page rank map

In [ ]:
plotMap("Page rank", pr, D.nodes, 0)

In [ ]:
print(nx.algorithms.assortativity.degree_assortativity_coefficient(G))
print(nx.algorithms.assortativity.degree_pearson_correlation_coefficient(G))

In [ ]:
print(nx.algorithms.assortativity.degree_mixing_matrix(G))

In [ ]:
#fig, ax = plt.subplots()

min_val, max_val = min(D.degree, key = lambda d: d[1])[1], max(D.degree, key = lambda d: d[1])[1]

mapping = {x: x for x in range(max_val//9)}

matrix = nx.algorithms.assortativity.degree_mixing_matrix(D, mapping=mapping)

# ax.matshow(matrix, cmap=plt.cm.Blues)

# for i in range(max_val//9-1):
#     for j in range(max_val//9-1):
#         c = '%.2f'%(matrix[j,i])
#         ax.text(i, j, str(c), va='center', ha='center')

f = plt.figure(figsize=(19, 15))
plt.matshow(matrix, fignum=f.number)
# plt.xticks(range(df.select_dtypes(['number']).shape[1]), df.select_dtypes(['number']).columns, fontsize=14, rotation=45)
# plt.yticks(range(df.select_dtypes(['number']).shape[1]), df.select_dtypes(['number']).columns, fontsize=14)
cb = plt.colorbar()
cb.ax.tick_params(labelsize=14)
plt.title('Correlation Matrix', fontsize=16)

## Studying the SI Model

In [ ]:
def plotMapSI(infected_dic, list_nodes):
    newAirportsDF = pd.DataFrame(columns=['id', 'name', 'city', 'country', 'IATA', 'ICAO', 'lat', 'lon', 'alt', 'timezone', 'DST', 'Tz', 'type', 'source'])

    # from the list given, add the nodes that have a higher centrality value than the threshold
    for node in list_nodes:
            x = airportsDF.loc[airportsDF['IATA'] == node]
            x['color'] = 1 if infected_dic[node] else 0
            x['markersize'] = 10
            newAirportsDF = pd.concat([newAirportsDF, x])

    geometry = [Point(xy) for xy in zip(newAirportsDF['lon'], newAirportsDF['lat'])]
    gdf = GeoDataFrame(newAirportsDF, geometry=geometry)

    # #this is a simple map that goes with geopandas
    world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
    gdf.plot(column="color", ax=world.plot(figsize=(20, 12)), marker='o', markersize="markersize", cmap='bwr') 
            #,legend=True, legend_kwds={"label": label, "orientation": "horizontal"})

In [ ]:
def plotSI(infected, susceptible):
    days = list(range(len(infected)))
    plt.figure(2)
    plt.plot(days, infected, label="Infected", color="red")
    plt.plot(days, susceptible, label="Susceptible", color="blue")
    plt.xlabel("Days")
    plt.ylabel("Number of people")
    plt.legend()
    plt.margins(x=0, y=0)
    plt.grid()

In [ ]:

# compute SI step
def SI_step(network):
    for node in network.nodes:
        if network.nodes[node]['infected'] and network.nodes[node]['active']:
            for neighbor in network.neighbors(node):
                if not network.nodes[neighbor]['infected']:
                    if np.random.random() < network.nodes[node]['beta']:
                        network.nodes[neighbor]['infected'] = True

# compute SI model
def SI_model(network, dc_sorted, nSteps, starterNode, theDay, airportPercentage):
    susceptible = []
    infected = []
    network.nodes[starterNode]['infected'] = True
    for step in range(nSteps):
        # remove nodes based on degree centrality at the Day
        if(step == theDay):
            # get the top 20% of nodes based on degree centrality
            closedAirports = dc_sorted[0:int(len(dc_sorted)*airportPercentage)]
            # remove the nodes that are not in the top 20%
            for node in network.nodes:
                if node in closedAirports:
                    network.nodes[node]['beta'] = 0.0001

        susceptible.append(len([node for node in network.nodes if not network.nodes[node]['infected']]))
        infected.append(len([node for node in network.nodes if network.nodes[node]['infected']]))
        SI_step(network)
    
    return susceptible, infected

def full_SI_model(graph, degree_centrality, nSteps, starterNode, theDay, airportPercentage):
    network = graph.copy()
    # get largest strongly connected component
    sccs = nx.strongly_connected_components(network)
    sccs_graphs = (network.subgraph(c) for c in sccs)
    network = list(sccs_graphs)[0]

    # sorted degree centrality
    dc_sorted = sorted(degree_centrality, key=dc.get, reverse=True)

    # add infection and recovery rate to nodes
    for node in network.nodes:
        network.nodes[node]['beta'] = 0.01
        network.nodes[node]['infected'] = False
        network.nodes[node]['active'] = True

    susceptible, infected = SI_model(network, dc_sorted, nSteps, starterNode, theDay, airportPercentage)
    return susceptible, infected, network



In [ ]:
# model without preventive measures
susceptible, infected, network = full_SI_model(G, dc, 100, 'LIS', 0, 0)
days = list(range(len(infected)))

infected_dic = {node:network.nodes[node]['infected'] for node in network.nodes}
#plt.figure(1)
#plotMapSI(infected_dic, network.nodes.keys())
print(infected[len(infected)-1])
print(susceptible[len(susceptible)-1])

# model with preventive measures
susceptible_prev, infected_prev, network_prev = full_SI_model(G, dc, 100, 'LIS', 7, 0.02)
infected_dic_prev = {node:network_prev.nodes[node]['infected'] for node in network_prev.nodes}

plt.figure(1)
plt.plot(days, infected, label="Infected", color="red")
plt.plot(days, susceptible, label="Susceptible", color="blue")
plt.plot(days, infected_prev, label="Infected (prev. measures)", color="red", linestyle='dashed')
plt.plot(days, susceptible_prev, label="Susceptible (prev. measures)", color="blue", linestyle='dashed')
plt.xlabel("Days")
plt.ylabel("Number of people")
plt.legend()
plt.margins(x=0, y=0)
plt.grid()

#plt.figure(3)
#plotMapSI(infected_dic_prev, network.nodes.keys())

plt.figure(2)
plt.plot(days, infected, label="Infected", color="red")
plt.plot(days, susceptible, label="Susceptible", color="blue")
plt.xlabel("Days")
plt.ylabel("Number of people")
plt.legend()
plt.margins(x=0, y=0)
plt.grid()

plt.figure(3)
plt.plot(days, infected_prev, label="Infected", color="red")
plt.plot(days, susceptible_prev, label="Susceptible", color="blue")
plt.xlabel("Days")
plt.ylabel("Number of people")
plt.legend()
plt.margins(x=0, y=0)
plt.grid()

In [ ]:
print(infected_prev[len(infected)-1])
print(susceptible_prev[len(susceptible)-1])